# A/B Email Campaign Testing Project
## Data Cleaning

Import Libraries

In [4]:
import numpy as np
import pandas as pd

Loading Dataset

In [7]:
# Load the dataset and parse dates in proper format
df = pd.read_csv('../data/raw/email_campaign_data.csv', parse_dates=['timestamp'])

View Data

In [8]:
df.head()

,user_id,variant,send_time,content_layout,account_age,feature_usage,user_segment,open,click,convert,timestamp,device
0,1,A,afternoon,text-heavy,216.0,48.0,active,1.0,1.0,0.0,2023-04-01 16:04:01,desktop
1,2,B,morning,text-heavy,1.0,20.0,new,1.0,1.0,0.0,2023-04-14 09:26:41,desktop
2,3,A,morning,balanced,357.0,47.0,active,0.0,0.0,0.0,2023-04-14 08:32:52,desktop
3,4,A,evening,balanced,262.0,33.0,active,1.0,1.0,0.0,2023-04-03 20:44:07,mobile
4,5,A,morning,visual-heavy,73.0,8.0,active,0.0,0.0,0.0,2023-04-12 11:52:06,desktop


In [11]:
print("Rows, Columns")
df.shape

Rows, Columns


(515000, 12)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515000 entries, 0 to 514999
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   user_id         515000 non-null  int64         
 1   variant         515000 non-null  object        
 2   send_time       515000 non-null  object        
 3   content_layout  515000 non-null  object        
 4   account_age     504698 non-null  float64       
 5   feature_usage   509837 non-null  float64       
 6   user_segment    512443 non-null  object        
 7   open            513681 non-null  float64       
 8   click           515000 non-null  float64       
 9   convert         515000 non-null  float64       
 10  timestamp       515000 non-null  datetime64[ns]
 11  device          515000 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(5)
memory usage: 47.1+ MB
